# 解析 站长素材

In [ ]:
# （1）请求对象的定制
# （2）获取网页的源码
# （3）下载

# 需求：下载前十页的图片 
# https://sc.chinaz.com/tupian/qinglvtupian.html
# https://sc.chinaz.com/tupian/qinglvtupian_2.html
# https://sc.chinaz.com/tupian/qinglvtupian_page.html

In [ ]:
import urllib.request
from lxml import etree
import os

def create_request(page):
    if page == 1:
        url = 'https://sc.chinaz.com/tupian/qinglvtupian.html'
    else:
        url = 'https://sc.chinaz.com/tupian/qinglvtupian_' + str(page) + '.html'
    
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'cookie': 'cz_statistics_visitor=5f44c562-7c22-a6a8-571b-19048cb8f903; Hm_lvt_398913ed58c9e7dfe9695953fb7b6799=1733377217; HMACCOUNT=52AB44D21035A831; _clck=avqsie%7C2%7Cfrg%7C0%7C1800; _clsk=ra7x14%7C1733377218810%7C1%7C1%7Cu.clarity.ms%2Fcollect; Hm_lpvt_398913ed58c9e7dfe9695953fb7b6799=1733381448'
    }
    request = urllib.request.Request(url=url, headers=headers)
    return request

def get_content(request):
    response = urllib.request.urlopen(request)
    content = response.read().decode('utf-8')
    # print(content)
    return content

def down_load(content, page):
    # 下载图片
    # urllib.request.urlretrieve('图片地址', '文件名')
    
    tree = etree.HTML(content)
    
    # 可以在浏览器通过xpath插件获取xpath路径
    # 但有时候爬取的源码和浏览器看到的不一样，需要根据content源码来定位
    # name_list = tree.xpath('//div[@class="container"]//a/img/@alt')
    name_list = tree.xpath('//div[@class="item"]/img/@alt')
    src_list = tree.xpath('//div[@class="item"]/img/@data-original')
    # 有的时候 len(src_list)会为0
    # 这是因为一般涉及图片的网站都会进行懒加载，只有真正浏览到才会变为src，之前是src2
    # src_list = tree.xpath('//div[@class="item"]/img/@src2')
    print(len(name_list), len(src_list))


    os.makedirs('./loveImg', exist_ok=True)
    for i in range(len(name_list)):
        name = name_list[i]
        src = src_list[i]
        url = 'https:' + src
        urllib.request.urlretrieve(url=url, filename='./loveImg/' + name + '.jpg')

if __name__ == '__main__':
    start_page = int(input('请输入起始页码：'))
    end_page = int(input('请输入结束页码：'))

    for page in range(start_page, end_page + 1):
        # （1）请求对象的定制
        request = create_request(page)
        # （2）获取网页的源码
        content = get_content(request)
        # （3）下载
        down_load(content, page)